In [1]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker
from pyecharts.globals import ThemeType
from pyecharts.globals import CurrentConfig
from pyecharts.datasets import register_url
CurrentConfig.ONLINE_HOST = "http://127.0.0.1:8000/assets/"
register_url("http://127.0.0.1:8000/")
from pyecharts.datasets import register_files
register_files({"mytheme": ["themes/mytheme", "js"]})

In [2]:
from pyecharts.datasets import register_url
data_pair = [['华中',100]]
# 注册本地服务器的地图数据源
register_url("http://127.0.0.1:8000/")
from pyecharts.charts import Map
maps = (Map(init_opts=opts.InitOpts(theme=ThemeType.DARK,bg_color='#404a59'))
    .add(series_name='销量',data_pair=data_pair,maptype='中国七大区',
        is_map_symbol_show=False,  # 关闭地图上的点
        label_opts=opts.LabelOpts(is_show=False)
        )
       )
maps.render_notebook()

In [8]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sys
print('Python version:', sys.version)
print('Pandas version:', pd.__version__)
print('Numpy version:', np.__version__)
print('Matplotlib version:', matplotlib.__version__)

Python version: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
Pandas version: 2.2.2
Numpy version: 1.26.4
Matplotlib version: 3.8.4


In [22]:
import os
def get_target_data(filepath, mapname = ''):
    target_data = """{
            "type": "FeatureCollection",
            "features": []}"""
    target_data = json.loads(target_data)
    list_region = os.listdir(filepath)
    name_list = []
    for lr in list_region:
        with open(f"{filepath}/{lr}", 'r', encoding='utf-8') as f:
            temp = json.load(f)
            temp = temp['features']
            for tp in temp:
                name = tp['properties']['name']
                # 将area加入tp节点并传入target_data
                tp['properties']['area'] = mapname
                target_data['features'].append(tp)
                name_list.append((name, np.random.randint(100, 5000)))
    return target_data

In [23]:
# json地图数据的构建
target_data = get_target_data('./地图资源/山河四省/',mapname='山河四省')
map=(Map()
    .add('山河四省',data_pair=[['河南省',100]],maptype='中国城市',is_map_symbol_show=False)
    .add_js_funcs(f"echarts.registerMap('中国城市', {target_data});")
    .set_global_opts(visualmap_opts=opts.VisualMapOpts(max_=100000,min_=20000))
    )
map.render_notebook()

In [26]:
import json

with open('./地图资源/陕甘宁-黑吉辽.geojson','r', encoding='utf-8') as f:
    target_data = json.loads(f.read())
    

map=(Map()
    .add_js_funcs(f"echarts.registerMap('陕甘宁-黑吉辽', {target_data});")
    .add('山河四省',data_pair=[['陕西省',100]],maptype='陕甘宁-黑吉辽',is_map_symbol_show=False)
    .set_global_opts(visualmap_opts=opts.VisualMapOpts(max_=100000,min_=20000))
    )
map.render_notebook()

In [84]:
target_data = get_target_data('./地图资源/陕甘宁/','陕甘宁')

In [29]:
map=(Map()
    .add_js_funcs(f"echarts.registerMap('陕甘宁-黑吉辽', {target_data});")
    .add('山河四省',data_pair=[['陕西省',100]],maptype='陕甘宁-黑吉辽',is_map_symbol_show=False)
    .set_global_opts(visualmap_opts=opts.VisualMapOpts(max_=100000,min_=20000))
    )
map.render_notebook()

In [49]:

zs = [z['geometry']['coordinates'][0][0] for z in target_data['features']]
len(zs)

3

In [71]:
len(list(itertools.chain(*zs)))

4494

In [110]:
target_data = get_target_data('./地图资源/陕甘宁/','陕甘宁')
# target_data['features'].clear()

In [109]:
target_data

{'type': 'FeatureCollection', 'features': []}

In [91]:
tb = pd.DataFrame(list(itertools.chain(*zs)))
tb.drop_duplicates(keep=False)

,0,1
1,106.468526,36.620424
2,106.491322,36.628789
3,106.495065,36.646396
4,106.494324,36.667301
5,106.491052,36.685734
...,...,...
4488,110.502688,34.345174
4489,110.489449,34.355280
4490,110.473734,34.393207
4491,110.475922,34.409159


In [113]:
import itertools
result = pd.DataFrame(list(itertools.chain(*zs))).drop_duplicates(keep=False).values.tolist()
result.append(result[0])
target_data['features'].clear()
dicts = {'type': 'Feature',
  'properties': {'adcode': 640000,
   'name': '陕甘宁',
   'center': [106.278179, 38.46637],
   'centroid': [106.169866, 37.291332],
   'childrenNum': 0,
   'level': 'province',
   'acroutes': [100000],
   'parent': {'adcode': 100000},
   'area': '陕甘宁'},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[result]]
              }
        }
target_data['features'].append(dicts)

In [117]:
maps=(Map()
    .add_js_funcs(f"echarts.registerMap('陕甘宁-黑吉辽', {target_data});")
    .add('山河四省',data_pair=[['陕西省',100]],maptype='陕甘宁-黑吉辽',is_map_symbol_show=False)
    .set_global_opts(visualmap_opts=opts.VisualMapOpts(max_=100000,min_=20000))
    )
maps.render_embed()